In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE46449"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE46449"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE46449.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE46449.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE46449.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from Patients with Bipolar (BP) Disorder and Matched Control Subjects"
!Series_summary	"There are currently no biological tests that differentiate patients with bipolar disorder (BPD) from healthy controls. While there is evidence that peripheral gene expression differences between patients and controls can be utilized as biomarkers for psychiatric illness, it is unclear whether current use or residual effects of antipsychotic and mood stabilizer medication drives much of the differential transcription. We therefore tested whether expression changes in first-episode, never-medicated bipolar patients, can contribute to a biological classifier that is less influenced by medication and could potentially form a practicable biomarker assay for BPD."
!Series_summary	"We employed microarray technology to measure global leukocyte gene expression in first-episode (n=3) and currently medicated BPD patients (n=26), and matched healthy control

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Any, Callable, Dict, Optional
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarray technology
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary:
# - trait: Key 1 contains 'genotype: bipolar patient', 'genotype: control subject'
# - age: Key 2 contains various age values
# - gender: Key 3 contains only 'gender: male', which seems to be a constant (only males in study)
trait_row = 1
age_row = 2
gender_row = None  # Only one value "male" in the entire dataset, so gender is not a useful variable

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert bipolar disorder trait data to binary values."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'bipolar' in value or 'bp' in value:
        return 1
    elif 'control' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age data to continuous values."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender data to binary values (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
# We have trait_row set, so trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we will extract clinical features
if trait_row is not None:
    # Assuming 'clinical_data' is already available from previous steps
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM1130402': [1.0, 63.0], 'GSM1130403': [1.0, 63.0], 'GSM1130404': [0.0, 43.0], 'GSM1130405': [0.0, 43.0], 'GSM1130406': [1.0, 40.0], 'GSM1130407': [1.0, 40.0], 'GSM1130408': [0.0, 28.0], 'GSM1130409': [0.0, 35.0], 'GSM1130410': [0.0, 35.0], 'GSM1130411': [1.0, 40.0], 'GSM1130412': [1.0, 40.0], 'GSM1130413': [1.0, 41.0], 'GSM1130414': [1.0, 41.0], 'GSM1130415': [0.0, 27.0], 'GSM1130416': [0.0, 27.0], 'GSM1130417': [0.0, 33.0], 'GSM1130418': [0.0, 33.0], 'GSM1130419': [1.0, 31.0], 'GSM1130420': [1.0, 31.0], 'GSM1130421': [0.0, 26.0], 'GSM1130422': [0.0, 26.0], 'GSM1130423': [0.0, 27.0], 'GSM1130424': [0.0, 27.0], 'GSM1130425': [0.0, 29.0], 'GSM1130426': [0.0, 42.0], 'GSM1130427': [0.0, 42.0], 'GSM1130428': [0.0, 28.0], 'GSM1130429': [0.0, 28.0], 'GSM1130430': [0.0, 27.0], 'GSM1130431': [0.0, 27.0], 'GSM1130432': [0.0, 37.0], 'GSM1130433': [0.0, 37.0], 'GSM1130434': [0.0, 25.0], 'GSM1130435': [0.0, 25.0], 'GSM1130436': [0.0, 36.0], 'GSM1130437': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Bipolar_disorder/GSE46449/GSE46449_series_matrix.txt.gz


Gene data shape: (54675, 88)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the data
# These identifiers (like '1007_s_at', '1053_at') are Affymetrix probe IDs from microarray data
# They are not standard human gene symbols and need to be mapped to proper gene symbols
# Affymetrix probe IDs typically have this format with '_at' suffix

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if IDs in gene_data match IDs in annotation
print("\nComparing gene data IDs with annotation IDs:")
print("First 5 gene data IDs:", gene_data.index[:5].tolist())
print("First 5 annotation IDs:", gene_annotation['ID'].head().tolist())

# Properly check for exact ID matches between gene data and annotation
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'].astype(str))
matching_ids = gene_data_ids.intersection(annotation_ids)
id_match_percentage = len(matching_ids) / len(gene_data_ids) * 100 if len(gene_data_ids) > 0 else 0

print(f"\nExact ID match between gene data and annotation:")
print(f"Matching IDs: {len(matching_ids)} out of {len(gene_data_ids)} ({id_match_percentage:.2f}%)")

# Check which columns might contain gene symbols for mapping
potential_gene_symbol_cols = [col for col in gene_annotation.columns 
                             if any(term in col.upper() for term in ['GENE', 'SYMBOL', 'NAME'])]
print(f"\nPotential columns for gene symbols: {potential_gene_symbol_cols}")

# Check if the identified columns contain non-null values
for col in potential_gene_symbol_cols:
    non_null_count = gene_annotation[col].notnull().sum()
    non_null_percent = non_null_count / len(gene_annotation) * 100
    print(f"Column '{col}': {non_null_count} non-null values ({non_null_percent:.2f}%)")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

Column 'Species Scientific Name': 54675 non-null values (1.12%)
Column 'Gene Title': 44680 non-null values (0.92%)
Column 'Gene Symbol': 45782 non-null values (0.94%)
Column 'ENTREZ_GENE_ID': 44134 non-null values (0.91%)
Column 'Gene Ontology Biological Process': 34524 non-null values (0.71%)


Column 'Gene Ontology Cellular Component': 36404 non-null values (0.75%)
Column 'Gene Ontology Molecular Function': 35177 non-null values (0.72%)


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain the probe IDs and gene symbols
# Based on annotation data preview, 'ID' column contains probe IDs same as gene_data.index
# 'Gene Symbol' column contains the human gene symbols

# 2. Create gene mapping dataframe by extracting these columns
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Generated mapping dataframe with shape: {mapping_df.shape}")
print("Mapping sample (first 5 rows):")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First 5 genes and their expression values:")
print(gene_data.head())

# Save the processed gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Generated mapping dataframe with shape: (45782, 2)
Mapping sample (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Converted gene expression data shape: (21278, 88)
First 5 genes and their expression values:
          GSM1130402  GSM1130403  GSM1130404  GSM1130405  GSM1130406  \
Gene                                                                   
A1BG        6.896147    6.542689    6.681429    6.445465    6.835954   
A1BG-AS1    4.728832    5.005751    4.586927    4.583834    4.649060   
A1CF        7.523571    7.664901    7.683107    7.740790    7.422415   
A2M         8.770531    9.348701    9.109397    8.979407    9.165433   
A2M-AS1     5.495042    5.413046    5.663845    5.494446    6.180433   

          GSM1130407  GSM1130408  GSM1130409  GSM1130410  GSM1130411  ...  \
Gene                                                     

Gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE46449.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First, let's reload the clinical_data to make sure we're using the correct data
# Note: The sample characteristics dictionary from step 1 showed:
# {1: ['genotype: bipolar patient', 'genotype: control subject'], 2: ['age: 63', ...], 3: ['gender: male']}

def convert_trait(value):
    """Convert bipolar disorder trait data to binary values."""
    if not isinstance(value, str):
        return None
    
    value = value.split(": ")[-1].strip().lower()
    if "bipolar" in value:
        return 1  # Has bipolar disorder
    elif "control" in value:
        return 0  # Control
    return None

def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# Get clinical data using the correct row index identified in step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,  # Using trait_row from step 2
    convert_trait=convert_trait,
    age_row=age_row,      # Using age_row from step 2
    convert_age=convert_age,
    gender_row=gender_row, # Using gender_row from step 2
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from peripheral blood leukocytes of bipolar disorder patients and matched control subjects."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (19845, 88)


Normalized gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE46449.csv
Selected clinical data shape: (2, 88)
Clinical data preview:
{'GSM1130402': [1.0, 63.0], 'GSM1130403': [1.0, 63.0], 'GSM1130404': [0.0, 43.0], 'GSM1130405': [0.0, 43.0], 'GSM1130406': [1.0, 40.0], 'GSM1130407': [1.0, 40.0], 'GSM1130408': [0.0, 28.0], 'GSM1130409': [0.0, 35.0], 'GSM1130410': [0.0, 35.0], 'GSM1130411': [1.0, 40.0], 'GSM1130412': [1.0, 40.0], 'GSM1130413': [1.0, 41.0], 'GSM1130414': [1.0, 41.0], 'GSM1130415': [0.0, 27.0], 'GSM1130416': [0.0, 27.0], 'GSM1130417': [0.0, 33.0], 'GSM1130418': [0.0, 33.0], 'GSM1130419': [1.0, 31.0], 'GSM1130420': [1.0, 31.0], 'GSM1130421': [0.0, 26.0], 'GSM1130422': [0.0, 26.0], 'GSM1130423': [0.0, 27.0], 'GSM1130424': [0.0, 27.0], 'GSM1130425': [0.0, 29.0], 'GSM1130426': [0.0, 42.0], 'GSM1130427': [0.0, 42.0], 'GSM1130428': [0.0, 28.0], 'GSM1130429': [0.0, 28.0], 'GSM1130430': [0.0, 27.0], 'GSM1130431': [0.0, 27.0], 'GSM1130432': [0.0, 37.

Data shape after handling missing values: (88, 19847)
For the feature 'Bipolar_disorder', the least common label is '0.0' with 39 occurrences. This represents 44.32% of the dataset.
The distribution of the feature 'Bipolar_disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 27.0
  50% (Median): 34.0
  75%: 42.0
Min: 21.0
Max: 70.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Bipolar_disorder/GSE46449.csv
